Create the mobile traffic chunks (called samples) with the same length (5 minutes, 4 minutes, 3 minutes, 2 minutes, 1 minutes)

## Import packages

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# this path depends on your setup (need to contain sources folder)
# root_path = 'C:/MAppGraph/mappgraph/data'
root_path = 'D:/Omri'
# root_path = 'C:/MAppGraph/mappgraph/BOAdata'

## Config setting

In [ ]:
'''
Define set of hyper-parameters
List of tuples (duration, overlap)
'''
params = [(5, 3), (4, 2), (3, 1), (2, 0), (1, 0)]

## Create the mobile traffic chunks with the same length

In [ ]:
# folder that contains source data
# sources_folder = os.path.join(root_path, 'sources')
# sources_folder = 'C:/MAppGraph/mappgraph/data/sources'
sources_folder = 'D:/Omri/sources'
# sources_folder = 'C:/MAppGraph/mappgraph/BOAdata/d_hi_chrome'

In [ ]:
# loop over set of hyper-parameters
for duration, overlap in params:
  # folder that contain samples of one set of parameters
  param_folder = os.path.join(root_path, '%d_%d'%(duration, overlap))
  
  # check whether the data is already generated or not
  if not os.path.exists(param_folder):
    os.mkdir(param_folder)

    # create folder to contain samples
    samples_folder = os.path.join(param_folder, 'samples')
    os.mkdir(samples_folder)

    # loop over each app to generate samples
    for app in os.listdir(sources_folder):
      print('App: ', app)
      app_sources_folder = os.path.join(sources_folder, app)

      # create folder contain samples for each app
      app_samples_folder = os.path.join(samples_folder, app)
      if not os.path.exists(app_samples_folder):
        os.mkdir(app_samples_folder)

      for filename in os.listdir(app_sources_folder):
        print('Processing %s ...' % filename)
        index = 1

        file_path = os.path.join(app_sources_folder, filename)
        df = pd.read_csv(file_path, index_col=0)
        base = df['time'].iloc[0]
        end = df['time'].iloc[-1]
      
        while ((index - 1)*(duration - overlap) + duration)*60 + base < end:
          start_time = base + (index-1)*(duration - overlap)*60
          end_time = start_time + duration*60
          df_ = df[(df['time'] >= start_time) & (df['time'] <= end_time)].reset_index(drop=True)

          # save a sample as csv file
          sample_filename = "_".join(filename.split('_')[:-2]) + '_' + filename.split('_')[-2] + '_' + str(index) + '.csv'
          sample_path = os.path.join(app_samples_folder, sample_filename)
          df_.to_csv(sample_path, index=True)

          index += 1
    
      print('...................................................')